In [1]:
import pandas as pd
import numpy as np
import tensorflow as tf
import cv2


2026-02-02 15:31:38.693118: I tensorflow/core/platform/cpu_feature_guard.cc:210] This TensorFlow binary is optimized to use available CPU instructions in performance-critical operations.
To enable the following instructions: AVX2 FMA, in other operations, rebuild TensorFlow with the appropriate compiler flags.


In [49]:
df = pd.read_csv('coffee_beans.csv')
X = df["filepaths"]
y = df["class index"]


In [50]:
def load_images(x):
    images = []
    for path in x:
        img = cv2.imread(path)
        img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
        img = cv2.resize(img, (224, 224))
        img = img / 255.0  # normalize
    
        images.append(img)
    
    images = np.array(images, dtype=np.float32)
    return images



Split dataset into training, validation and test set. This split will be used to choose the best neural network architecture for the final model.

In [51]:
from sklearn.model_selection import train_test_split

X_train, x_, y_train, y_ = train_test_split(X , y, test_size=0.40, random_state=1)
X_cv, X_test, y_cv, y_test = train_test_split(x_ , y_, test_size=0.20, random_state=1)

del x_, y_

Load the images defined in the $x$ sets.

In [52]:
X_train = load_images(X_train)
X_cv = load_images(X_cv)
X_test = load_images(X_test)

In [42]:
# df_train = df[df["filepaths"].str.startswith("train/")]
# df_test = df[df["filepaths"].str.startswith("test/")]

# X_train, y_train = load_images(df_train)
# X_test, y_test = load_images(df_test)


In [60]:
tf.keras.utils.set_random_seed(42) # seed for reproducibility

from tensorflow.keras import Sequential
from tensorflow.keras.layers import Dense, Flatten

model_1 = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=25, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI_1")

model_2 = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=15, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI_2")

model_3 = Sequential([
    Flatten(input_shape=(224, 224, 3)), # ??? have to research
    Dense(units=5, activation='relu'),
    Dense(units=15, activation='relu'),
    Dense(4, activation='linear'), # since the from_logits is used in the loss function
], name="CoffeeRoastAI_3")

nn_models = [model_1, model_2, model_3]

In [71]:
from sklearn.metrics import log_loss

nn_train_cross_entropy = []
nn_cv_cross_entropy = []

for model in nn_models:
    model.compile(
        optimizer=tf.keras.optimizers.Adam(learning_rate=0.001), # gradient descent optimatiation
        loss= SparseCategoricalCrossentropy(from_logits=True),
        metrics=['accuracy'], # ??? have to research
    )
    print(f"Training {model.name}...")
    
    model.fit(X_train, y_train, epochs=10)
    print("Done\n")

    # Instantiate loss function
    loss_fn = tf.keras.losses.SparseCategoricalCrossentropy()

    # Record the training Log Losses
    yhat = model.predict(X_train)
    pred_class = np.argmax(yhat, axis=1)[0]
    train_cross_entropy = loss_fn(y_train, yhat)
    nn_train_cross_entropy.append(train_cross_entropy)

    # Record the cross validation Log Losses
    yhat = model.predict(X_cv)
    pred_class = np.argmax(yhat, axis=1)[0]
    cv_cross_entropy = loss_fn(y_cv, yhat)
    nn_cv_cross_entropy.append(cv_cross_entropy)

Training CoffeeRoastAI_1...
Epoch 1/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 2s 36ms/step - accuracy: 0.2656 - loss: 1.7781
Epoch 2/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.2385 - loss: 1.3941
Epoch 3/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2385 - loss: 1.3916
Epoch 4/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2385 - loss: 1.3899
Epoch 5/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2385 - loss: 1.3887
Epoch 6/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2385 - loss: 1.3878
Epoch 7/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 36ms/step - accuracy: 0.2385 - loss: 1.3872
Epoch 8/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 37ms/step - accuracy: 0.2385 - loss: 1.3867
Epoch 9/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 40ms/step - accuracy: 0.2438 - loss: 1.3864
Epoch 10/10
30/30 ━━━━━━━━━━━━━━━━━━━━ 1s 39ms/step - accuracy: 0.2604 - loss: 1.3862
Done

30/30 ━━━━━━━━━━━━━━━━━━━━ 0s 8ms/step
16/16 ━━━━━━━━━━━━━━━━━━━━ 0s 10ms/step
Training CoffeeRoastAI_2...
Epoch 1/

In [73]:
print("RESULTS:")
for model_num in range(len(nn_train_cross_entropy)):
    print(
        f"Model {model_num+1}: Training MSE: {nn_train_cross_entropy[model_num]:.2f}, " +
        f"CV MSE: {nn_train_cross_entropy[model_num]:.2f}"
        ) 

RESULTS:
Model 1: Training MSE: 1.60, CV MSE: 1.60
Model 2: Training MSE: 4.35, CV MSE: 4.35
Model 3: Training MSE: 6.89, CV MSE: 6.89


Because softmax is integrated into the output layer the output s a vector of probabilities.

Make predictions

In [74]:
logits = model(X_train) # outpus z_1, .., z_n and not a_1, .., a_n
f_x = tf.nn.softmax(logits) # map the z result to softmax function
f_x

<tf.Tensor: shape=(960, 4), dtype=float32, numpy=
array([[0.24802086, 0.23884034, 0.2603363 , 0.25280246],
       [0.24802086, 0.23884034, 0.2603363 , 0.25280246],
       [0.24802086, 0.23884034, 0.2603363 , 0.25280246],
       ...,
       [0.24802086, 0.23884034, 0.2603363 , 0.25280246],
       [0.24802086, 0.23884034, 0.2603363 , 0.25280246],
       [0.24802086, 0.23884034, 0.2603363 , 0.25280246]],
      shape=(960, 4), dtype=float32)>

In [75]:
class_names = {
    0: "Dark",
    1: "Green",
    2: "Light",
    3: "Medium"
}


In [76]:
IMG_SIZE = 224

img = cv2.imread("test/Medium/medium (13).png")
img = cv2.cvtColor(img, cv2.COLOR_BGR2RGB)
img = cv2.resize(img, (IMG_SIZE, IMG_SIZE))
img = img / 255.0

img = np.expand_dims(img, axis=0)  # shape (1, 224, 224, 3)

pred = model.predict(img)
pred_class = np.argmax(pred, axis=1)[0]

print("Predicted:", class_names[pred_class])
print("Class probabilities (Dark, Green, Light, Medium):", pred[0])


1/1 ━━━━━━━━━━━━━━━━━━━━ 0s 70ms/step
Predicted: Light
Class probabilities (Dark, Green, Light, Medium): [-0.00444375 -0.04216134  0.04401769  0.01465181]


Save the model

In [ ]:
model.save("coffee_roast_model.keras")